In [11]:
import theano
import theano.tensor as T
import numpy
 
x = T.fvector('x')
W = theano.shared(numpy.asarray([0.2, 0.7]), 'W')
y = (x * W).sum()
 
f = theano.function([x], y)
 
output = f([1.0, 1.0])
print(output)

0.8999999999999999


In [13]:
x = T.fvector('x')
target = T.fscalar('target')
 
W = theano.shared(numpy.asarray([0.2, 0.7]), 'W')
y = (x * W).sum()
 
cost = T.sqr(target - y)
gradients = T.grad(cost, W)
W_updated = W - (0.1 * gradients)
updates = [(W, W_updated)]
 
f = theano.function([x, target], y, updates = updates)
 
for i in range(10):
    output = f([1.0, 1.0], 20.0)
    print(output)

0.8999999999999999
8.540000113844872
13.124000136613844
15.874400122952459
17.524640098361967
18.514784073771477
19.10887045311546
19.46532227718082
19.67919336949542
19.80751602360941


In [14]:
a = theano.shared(numpy.asarray([[1.0,2.0],[3.0,4.0]]), 'a')
a.eval()

array([[ 1.,  2.],
       [ 3.,  4.]])

In [87]:
a.diagonal().eval()

array([ 1.,  4.])

In [15]:
c = theano.tensor.nnet.softmax(a)
c.eval()

array([[ 0.26894142,  0.73105858],
       [ 0.26894142,  0.73105858]])

http://www.marekrei.com/blog/theano-tutorial/

In [1]:
from theano_bpr.utils import load_data_from_movielens

In [2]:
training_data, users_to_index, items_to_index = load_data_from_movielens('http://files.grouplens.org/datasets/movielens/ml-100k/ua.base', 3)
testing_data, users_to_index, items_to_index = load_data_from_movielens('http://files.grouplens.org/datasets/movielens/ml-100k/ua.test', 3, users_to_index, items_to_index)

In [3]:
from theano_bpr import BPR

In [11]:
bpr = BPR(10, len(users_to_index.keys()), len(items_to_index.keys()))
bpr.train(training_data, epochs=50)

Generating 2495300 random training samples
Processed 2495000 ( 99.99% ) in 0.0066 seconds
Total training time 16.76 seconds; 6.714787e-06 per sample


In [12]:
bpr.test(testing_data)

Current AUC mean (900 samples): 0.91299


0.9129410191673194

# Implement

In [4]:
import sys
import numpy as np
import theano
import theano.tensor as T
from time import time
from collections import defaultdict

In [5]:
rank = 10
n_users = len(users_to_index)
n_items = len(items_to_index)
lambda_u = 0.0025
lambda_i = 0.0025
lambda_j = 0.00025
lambda_bias = 0.0
learning_rate = 0.05

n_iters = 50
batch_size = 1000

In [6]:
def _data_to_dict(data):
    data_dict = defaultdict(list)
    items = set()
    for user, item in data:
        data_dict[user].append(item)
        items.add(item)
    
    return data_dict, set(data_dict.keys()), items

In [7]:
# item rated by the user, unique user, item index
_train_dict, _train_users, _train_items = _data_to_dict(training_data)

# number of interactions * epochs
n_sgd_samples = len(training_data) * n_iters

In [8]:
def _uniform_user_sampling(n_samples):
    sampled_users = np.random.choice(n_users, size = n_samples)
    sgd_users = np.array(list(_train_users))[sampled_users]
    
    # for each randomly sampled user, generate the corresponding positive and negative item
    sgd_pos_items, sgd_neg_items = [], []
    for sgd_user in sgd_users:
        interacted_items = _train_dict[sgd_user]
        pos_item = interacted_items[np.random.choice(len(interacted_items))]
        sgd_pos_items.append(pos_item)

        neg_item = np.random.choice(n_items)
        while neg_item in _train_dict[sgd_user]:
            neg_item = np.random.choice(n_items)

        sgd_neg_items.append(neg_item)
        
    return sgd_users, sgd_pos_items, sgd_neg_items

In [9]:
sgd_users, sgd_pos_items, sgd_neg_items = _uniform_user_sampling(n_samples = n_sgd_samples)

In [13]:
u = T.lvector('u')
i = T.lvector('i')
j = T.lvector('j')

W = theano.shared(np.random.random((n_users, rank)).astype('float32'), name='W')
H = theano.shared(np.random.random((n_items, rank)).astype('float32'), name='H')
B = theano.shared(np.zeros(n_items).astype('float32'), name='B')

x_ui = T.dot(W[u], H[i].T).diagonal()
x_uj = T.dot(W[u], H[j].T).diagonal()
x_uij = B[i] - B[j] + x_ui - x_uj

# minimize the cost
reg_u = lambda_u * (W[u] ** 2).sum()
reg_i = lambda_i * (H[i] ** 2).sum()
reg_j = lambda_j * (H[j] ** 2).sum()
cost = reg_u + reg_i + reg_j - T.log(T.nnet.sigmoid(x_uij)).sum()

g_cost_W = T.grad(cost=cost, wrt=W)
g_cost_H = T.grad(cost=cost, wrt=H)
g_cost_B = T.grad(cost=cost, wrt=B)

updates = [ (W, W - learning_rate * g_cost_W), (H, H - learning_rate * g_cost_H), (B, B - learning_rate * g_cost_B) ]
train_model = theano.function(inputs=[u, i, j], outputs=cost, updates=updates)

In [14]:
# batch training
z = 0
t2 = t1 = time()
while (z+1)*batch_size < n_sgd_samples:
    train_model(
        sgd_users[z*batch_size: (z+1)*batch_size],
        sgd_pos_items[z*batch_size: (z+1)*batch_size],
        sgd_neg_items[z*batch_size: (z+1)*batch_size]
    )
    z += 1
    t2 = time()
    sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * z*batch_size/n_sgd_samples, t2 - t1))
    sys.stderr.flush()
    t1 = t2

Processed 2495000 ( 99.99% ) in 0.0064 seconds

In [18]:
def predict_user(user):
    w = W.get_value()
    h = H.get_value()
    b = B.get_value()
    user_pred = w[user].dot(h.T) + b 
    return user_pred

In [16]:
test_dict, test_users, test_items = _data_to_dict(testing_data)

In [17]:
auc = 0.0
for user in test_dict:
    n = 0
    user_auc = 0.0
    predictions = predict_user(user)

    # training dataset's interaction is the negative item for test set
    pos_items = test_dict[user]
    #neg_items = _train_dict[user]
    for pos_item in pos_items:
        if pos_item in _train_items:
            #for neg_item in neg_items:
            for neg_item in _train_items:
                if neg_item not in test_dict[user] and neg_item not in _train_dict[user]:
                    n += 1
                    if predictions[pos_item] > predictions[neg_item]:
                        user_auc += 1

    user_auc /= n
    auc += user_auc
    
auc / len(test_dict)

0.9160137687835515

In [26]:
auc_values = []

for user in test_dict:
    if user in _train_users:
        auc_for_user = 0.0
        n = 0
        predictions = predict_user(user)
        for pos_item in test_dict[user]:
            if pos_item in _train_items:
                # training dataset's interaction is the negative item for test set
                for neg_item in _train_items:
                    if neg_item not in test_dict[user] and neg_item not in _train_dict[user]:
                        n += 1
                        if predictions[pos_item] > predictions[neg_item]:
                            auc_for_user += 1
        if n > 0:
            auc_for_user /= n
            auc_values.append(auc_for_user)
            
np.mean(auc_values)

0.91475319760458429

In [ ]:
from urllib.request import urlopen
url = 'http://files.grouplens.org/datasets/movielens/ml-100k/ua.base'
threshold = 3 # rating for positive

raw_data = []
for line in urlopen(url).readlines():
    user, item, rating, timestamp = str(line, 'utf-8').split('\t')
    if int(rating) > threshold:
        raw_data.append((user, item))

In [ ]:
# This function will return a data array consisting
# of (user, item) tuples, a mapping from user ids to integers
# and a mapping from item ids to integers.
users_to_i = {}
items_to_i = {}

# coordinates that have positive interactions
data = []

u = 0
i = 0
for user, item in raw_data:
    if user not in users_to_i:
        users_to_i[user] = u
        u += 1
    
    if item not in items_to_i:
        items_to_i[item] = i
        i += 1
    
    data.append((users_to_i[user], items_to_i[item]))